# Spatial TX registration to blockface

Process
1. Register spatial TX barcode to its respective blockface image
    a. If blockface section doesn't exist, pick the closets section  
    b. If blockface set doesn't exist, duplicate image from closest set and rename to target set. (See readme for mapping)  
    c. Perform QC on registration from the outputted QC images  
    d. For registration that failed, this is due to a failure in the initial rotation search. A rotation is visually picked and the registration is re-run


In [ ]:
from hmba_stx_registration import Specimen, get_metadata_df, register_cells_to_blockface
from pathlib import Path

base_path = Path("/home/mike/workspace/data/macaque_brain/QM24.50.002")
sptx_path = base_path / "sptx"
barcodes_path = sptx_path / "xenium"
um_per_px = 20
mm_per_px = um_per_px / 1000
metadata_df = get_metadata_df(barcodes_path)

/home/mike/workspace/data/macaque_brain/QM24.50.002/sptx/xenium/QM24.50.002_section_metadata.csv is not a directory, skipping


### Perform registration between ST and Blockface

For each barcode:
-> Register to blockface  
**Inputs:**  
1. blockface image path  
2. Specimen object  
    a. cells table  
    b. specimen metadata  

**Outputs:**  
1. transforms_out_path: directory to store   affines in .npy format  
2. qc_path: directory to store plots to visualize registration for QC  


In [11]:
barcodes = metadata_df['barcode'].tolist()
for barcode in barcodes:
    specimen = Specimen(barcode, barcodes_path)
    print("Registering specimen:", specimen.specimen_name)
    bf_img_path_base = base_path / 'blocks' / specimen.slab_name / 'processed'
    bf_img_path = list(bf_img_path_base.glob(f"{specimen.specimen_set_name}*.png"))[0]
    transforms_path = sptx_path / "st2block_output_02162026"
    st2bf_affine = register_cells_to_blockface(specimen.cells_table,
                            bf_img_path,
                            specimen.specimen_name,
                            transforms_out_path=transforms_path,
                            qc_path=transforms_path / "qc",
                            gamma=0.3
                            )

Registering specimen: QM24.50.002.CX.45.05.01.02
Registering specimen: QM24.50.002.CX.52.01.03.02
Registering specimen: QM24.50.002.CX.47.01.05.02
Registering specimen: QM24.50.002.CX.48.02.01.02
Registering specimen: QM24.50.002.CX.48.06.03.02
Registering specimen: QM24.50.002.CX.47.06.01.02
Registering specimen: QM24.50.002.CX.50.03.03.02
Registering specimen: QM24.50.002.CX.48.02.05.03
Registering specimen: QM24.50.002.CX.45.01.01.02
Registering specimen: QM24.50.002.CX.46.01.05.02
Registering specimen: QM24.50.002.CX.46.05.05.02
Registering specimen: QM24.50.002.CX.52.01.05.02
Registering specimen: QM24.50.002.CX.47.02.04.02
Registering specimen: QM24.50.002.CX.45.01.03.02
Registering specimen: QM24.50.002.CX.45.05.03.02
Registering specimen: QM24.50.002.CX.51.01.07.03
Registering specimen: QM24.50.002.CX.46.04.05.02
Registering specimen: QM24.50.002.CX.47.03.03.02
Registering specimen: QM24.50.002.CX.46.06.03.02
Registering specimen: QM24.50.002.CX.49.03.01.02
Registering specimen

### Rerun registration for sets that failed QC by visually defining the rotation

In [13]:
setname_rotation = {
 'QM24.50.002.CX.47.04.01': 180,
 'QM24.50.002.CX.48.05.01': 180,
 'QM24.50.002.CX.48.04.03': 180,
 'QM24.50.002.CX.45.05.03': 45,  
 'QM24.50.002.CX.47.03.03': 180,
 'QM24.50.002.CX.50.03.01': 180,
 'QM24.50.002.CX.50.01.01': 180,
 'QM24.50.002.CX.50.05.03': 0,
 'QM24.50.002.CX.50.08.01': 90,
 'QM24.50.002.CX.50.08.05': 135,
 'QM24.50.002.CX.50.09.03': -170,
 'QM24.50.002.CX.51.01.07': 180
}

for setname, rotation in setname_rotation.items():
    for barcode in metadata_df.loc[metadata_df.specimen_set_name == setname].barcode.values:
            specimen = Specimen(barcode, barcodes_path)
            print("Registering specimen:", specimen.specimen_name)
            bf_img_path_base = base_path / 'blocks' / specimen.slab_name / 'processed'
            bf_img_path = list(bf_img_path_base.glob(f"{specimen.specimen_set_name}*.png"))[0]
            transforms_path = sptx_path / "st2block_output_02162026"
            st2bf_affine = register_cells_to_blockface(specimen.cells_table,
                                    bf_img_path,
                                    specimen.specimen_name,
                                    transforms_out_path=transforms_path,
                                    qc_path=transforms_path / "qc",
                                    gamma=0.3,
                                    rotation=rotation,
                                    )


Registering specimen: QM24.50.002.CX.47.04.01.02
Registering specimen: QM24.50.002.CX.48.05.01.02
Registering specimen: QM24.50.002.CX.48.04.03.02
Registering specimen: QM24.50.002.CX.45.05.03.02
Registering specimen: QM24.50.002.CX.47.03.03.02
Registering specimen: QM24.50.002.CX.47.03.03.03
Registering specimen: QM24.50.002.CX.50.03.01.01
Registering specimen: QM24.50.002.CX.50.01.01.02
Registering specimen: QM24.50.002.CX.50.05.03.02
Registering specimen: QM24.50.002.CX.50.08.01.02
Registering specimen: QM24.50.002.CX.50.08.05.02
Registering specimen: QM24.50.002.CX.50.09.03.02
Registering specimen: QM24.50.002.CX.51.01.07.03
